In [65]:
import pandas as pd
import psycopg2
import os
from sqlalchemy import create_engine, text, MetaData, Table

### STEP 1: Load data

#### STEP 1A: Loading Existing Data from base tables (Project, Source, Types)

In [66]:
db_params = {
    'host': os.getenv('DB_HOST') or 'localhost',
    'database': os.getenv('DB_NAME') or 'tbmc_db',
    'user': os.getenv('DB_USER') or 'tbmc_db_user',
    'password': os.getenv('DB_PASSWORD') or '123456',
    'table': os.getenv('DB_TABLE') or 'tbmc_db1',
    'port': os.getenv('DB_PORT') or '5432'
}

In [67]:
def connect_to_database(db_params):
    try:
        conn = psycopg2.connect(
            host=db_params['host'],
            database=db_params['database'],
            user=db_params['user'],
            password=db_params['password']
        )
        conn.set_session(autocommit=True)
        
        engine = create_engine(f"postgresql://{db_params['user']}:{db_params['password']}@{db_params['host']}:{db_params['port']}/{db_params['database']}")
        
        if conn and engine:
            print("Connection successful!")
            return conn, engine
        else:
            print("Connection failed.")

        
    except Exception as e:
        print(f"Error connecting to database: {e}")
        return None, None

conn, engine = connect_to_database(db_params)



Connection successful!


In [68]:
def existing_table_data():
    if engine:
        query_type = "SELECT * FROM type"
        query_source = "SELECT * FROM source"
        query_items = "SELECT * FROM items"

        item_type = pd.read_sql(query_type, engine)
        item_source = pd.read_sql(query_source, engine)
        item_items = pd.read_sql(query_items, engine)


        return item_type, item_source, item_items

    else:
        print("Failed to connect to the database.")

type_table, source_table, items_table = existing_table_data()

In [69]:
item_columns = list(items_table)

#### STEP 1A: Loading New Data

In [70]:
file_path = "../data/quotation_princing_analysis/"
file_name = "INDEPENDENT CIP EVAPORATOR CIR RETURN LINE GOING TO IS 400M3 TANK - INHOUSE.xlsx"
file_path_name = file_path + file_name
data = pd.read_excel(file_path_name, names=item_columns)

In [71]:
data.head(15)

,description,man_r,unit,qty,unit_cost,amount,project,type,date,source
0,mobilization,NaN,lot,1,50000.0,50000.0,INDEPENDENT CIP EVAPORATOR - CIR RETURN LINE G...,Mobilization/Demobilization & Housekeeping,NaN,INHOUSE
1,demobilization/housekeeping,NaN,lot,1,50000.0,50000.0,INDEPENDENT CIP EVAPORATOR - CIR RETURN LINE G...,Mobilization/Demobilization & Housekeeping,NaN,INHOUSE
2,cotton gloves,NaN,pair,480,27.0,12960.0,INDEPENDENT CIP EVAPORATOR - CIR RETURN LINE G...,Safety Provisions,NaN,INHOUSE
3,dust mask,NaN,bxs,5,277.0,1385.0,INDEPENDENT CIP EVAPORATOR - CIR RETURN LINE G...,Safety Provisions,NaN,INHOUSE
4,caution tape,NaN,roll,1,1384.0,1384.0,INDEPENDENT CIP EVAPORATOR - CIR RETURN LINE G...,Safety Provisions,NaN,INHOUSE
5,safety signages,NaN,lot,1,5000.0,5000.0,INDEPENDENT CIP EVAPORATOR - CIR RETURN LINE G...,Safety Provisions,NaN,INHOUSE
6,fire blanket,NaN,pcs,6,3500.0,21000.0,INDEPENDENT CIP EVAPORATOR - CIR RETURN LINE G...,Safety Provisions,NaN,INHOUSE
7,welding blanket,NaN,pcs,6,3500.0,21000.0,INDEPENDENT CIP EVAPORATOR - CIR RETURN LINE G...,Safety Provisions,NaN,INHOUSE
8,cip suit(16 person),NaN,set,34,2000.0,68000.0,INDEPENDENT CIP EVAPORATOR - CIR RETURN LINE G...,Safety Provisions,NaN,INHOUSE
9,construction board,NaN,lot,1,10000.0,10000.0,INDEPENDENT CIP EVAPORATOR - CIR RETURN LINE G...,Safety Provisions,NaN,INHOUSE


In [72]:
if engine:    
    # Append the DataFrame to the table named 'items' in the connected database
    data.to_sql('items', engine, if_exists='append', index=False)
    print("DataFrame appended to the database successfully!")
else:
    print("Failed to connect to the database.")


IntegrityError: (psycopg2.errors.UniqueViolation) duplicate key value violates unique constraint "items_pkey"
DETAIL:  Key (description, project, source)=(mobilization, INDEPENDENT CIP EVAPORATOR - CIR RETURN LINE GOING TO IS 400M3 TANK, INHOUSE) already exists.

[SQL: INSERT INTO items (description, man_r, unit, qty, unit_cost, amount, project, type, date, source) VALUES (%(description__0)s, %(man_r__0)s, %(unit__0)s, %(qty__0)s, %(unit_cost__0)s, %(amount__0)s, %(project__0)s, %(type__0)s, %(date__0)s, %(source__ ... 13660 characters truncated ... 6)s, %(unit_cost__86)s, %(amount__86)s, %(project__86)s, %(type__86)s, %(date__86)s, %(source__86)s)]
[parameters: {'source__0': 'INHOUSE', 'type__0': 'Mobilization/Demobilization & Housekeeping', 'project__0': 'INDEPENDENT CIP EVAPORATOR - CIR RETURN LINE GOING TO IS 400M3 TANK', 'man_r__0': None, 'description__0': 'mobilization', 'unit__0': 'lot', 'qty__0': 1, 'date__0': None, 'amount__0': 50000.0, 'unit_cost__0': 50000.0, 'source__1': 'I

### STEP 2: Final cleaning

### STEP 3: Add project name to "Project" table

#### STEP 3A: Connect to the database

#### STEP 3B: Upload data

#### STEP 3C: Close Connection

In [73]:
conn.close()

In [74]:
if conn:
    conn.close()
    print("Connection closed.")

if engine:
    engine.dispose()
    print("Engine disposed.")

Connection closed.
Engine disposed.
